In [424]:
import pandas as pd
import numpy as np
import ast

In [425]:
top_5_up_df = pd.read_csv('Data/top_5_1980s_up.csv')
top_5_up_df.rename(columns={'Unnamed: 0':'id'}, inplace=True)
top_5_up_df.reset_index(inplace=True)
top_5_up_df.drop('index', axis=1, inplace=True)
top_5_up_df.set_index(['id','name'], inplace=True)
#top_5_up_df.head()
#top_5_df.shape

In [426]:
top_5_down_df = pd.read_csv('Data/top_5_1980s_down.csv')
top_5_down_df.rename(columns={'Unnamed: 0':'id'}, inplace=True)
top_5_down_df.reset_index(inplace=True)
top_5_down_df.drop('index', axis=1, inplace=True)
top_5_down_df.set_index(['id','name'], inplace=True)
#top_5_down_df.head()
#top_5_df.shape

In [427]:
top_5_df = pd.concat([top_5_up_df, top_5_down_df])
#top_5_df.shape

In [428]:
indices = pd.Series(top_5_df.index.get_level_values(1))
#indices

In [429]:
#TRY TO GET DROP DOWN LIST WITH ARTIST NAME FOR SONGS WITH SAME NAME
def num_of_artists(name):
    artists = []
    name = name.lower()
    for i in range(len(indices[indices==name])):
        idx = indices[indices == name].index[i]
        artists.append(top_5_df.iloc[idx][0])
    return artists

In [430]:
def recommendations(name, artists):
    #INPUT A SONG RETURN
    name = name.lower()
    ind = 0
    for i in range(len(indices[indices == name])):
        if(top_5_df.iloc[indices[indices == name].index[i]][0] == artists):
            ind = i
    idx = indices[indices == name].index[ind]
    #print(top_5_df.iloc[idx][0])
    
    top = top_5_df.iloc[idx]
    top_5_dict = ast.literal_eval(top.top_5)
    return top_5_dict

In [431]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.core.display import display, HTML
from IPython.display import display, clear_output

In [432]:
%%html
<style>

.box_style{
    width:100%;
    height:100%;
}

.btn_style{
    background-color:green;
    color:black;
}
</style>

In [433]:
song_name = widgets.Text(placeholder='Please enter a song')

artists = num_of_artists(song_name.value)

artist = widgets.Dropdown(
            options=artists,
            description='Artists Name',
            disbaled=False)

In [434]:
def update_song(change):
    song_name.value = change['new']
    artists = num_of_artists(song_name.value)
    artist.options = artists
song_name.observe(update_song, 'value')


In [435]:
get_top_5 = widgets.Button(
                description='Find recommended songs',
                tooltip='Find Song',
                layout=widgets.Layout(width='30%', height='50px'),
                style={'description_width': 'intitial'}
            )
        
get_top_5.add_class("btn_style")

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        try:
            top_5 = recommendations(str(song_name.value), str(artist.value))
            display(HTML(f'<h3>Here are the top 5 recommended songs for {song_name.value} by {artist.value}</h3>'))
            for key, value in top_5.items():
                url = f'https://open.spotify.com/track/{key}'
                display(HTML('<div style=font-size:large>Song Name: <a href={}> {}</a> by: {}</div>'.format(url, value[0], value[1])))
        except IndexError:
            display(HTML(f'<h3>Sorry Song Not Found! Please Enter in another Song</h3>'))
        
get_top_5.on_click(on_button_clicked)

btn_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')
hbox_result = widgets.HBox([song_name, artist])
vbox_result = widgets.VBox([hbox_result, get_top_5, output], layout=btn_layout)

In [436]:
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

In [437]:
text_0 = widgets.HTML(value='<h1 style=color:green>Jacob\'s Jukebox<h1>')
text_1 = widgets.HTML(value='<h2 style=color:green>Enter in a song to find the top 5 recommended songs!')

vbox_text = widgets.VBox([text_0, text_1], layout=box_layout)
vbox_output=widgets.VBox([vbox_text, vbox_result])

In [438]:
page = widgets.HBox([vbox_output], layout=box_layout)
page.add_class("box_style")
display(page)